In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [7]:
query = '3 곱하기 5는?'

llm_with_tools = llm.bind_tools([add, multiply])

result = llm_with_tools.invoke(query)

result.tool_calls

# [{'name': 'multiply',
#   'args': {'a': 3, 'b': 5},
#   'id': 'call_vemS0UHHfNKkdHCbI01fgUwt',
#   'type': 'tool_call'}]

tool_message = multiply.invoke(result.tool_calls[0])
tool_message


ToolMessage(content='15', name='multiply', tool_call_id='a1cb0e01-0e82-46bf-86f3-dd49eb5bbb96')

In [ ]:
from typing import Sequence

from langchain_core.messages import AnyMessage, HumanMessage

# 1. llm에 tools를 bind
# 2. llm_with_tools와 tool로 message_list를 채워넣음
# 3. llm_with_tools를 message_list로 invoke
human_message = HumanMessage(query)
message_list: Sequence[AnyMessage] = [human_message] 

ai_message = llm_with_tools.invoke(message_list)
message_list.append(ai_message)

# ai_message.tool_calls[0]에는 args 정보가 들어있다.
tool_message = multiply.invoke(ai_message.tool_calls[0])
message_list.append(tool_message)

# message_list
# 1. HumanMessage: query
# 2. AIMessage: 사용해야하는 tool과 args 정보가 담겨있다.
# 3. TollMessage: 결과값이 담겨있다.

llm_with_tools.invoke(message_list)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 398, 'total_tokens': 410, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-mini-250422', 'system_fingerprint': None, 'id': 'dd65480e-89c1-4c9a-9249-65b45ff5b6fe', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--630dfeb6-64d5-4f42-b4b6-b902d74ff132-0', usage_metadata={'input_tokens': 398, 'output_tokens': 12, 'total_tokens': 410, 'input_token_details': {}, 'output_token_details': {}})